In [1]:
from utils import NerDataset

dev_dataset = NerDataset('./processed/processed_dev_bio.txt')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
len(dev_dataset.sents)

909

In [4]:
train_dataset = NerDataset('./processed/processed_training_bio.txt')
len(train_dataset)

6306

In [5]:
test_dataset[0]

('[CLS] ， 患 者 3 月 前 因 “ 直 肠 癌 ” 于 在 我 院 于 全 麻 上 行 直 肠 癌 根 治 术 （ D I X O N 术 ） ， 手 术 过 程 顺 利 ， 术 后 给 予 抗 感 染 及 营 养 支 持 治 疗 ， 患 者 恢 复 好 ， 切 口 愈 合 良 好 。 ， 术 后 病 理 示 ： 直 肠 腺 癌 （ 中 低 度 分 化 ） ， 浸 润 溃 疡 型 ， 面 积 3 . 5 * 2 C M ， 侵 达 外 膜 。 双 端 切 线 另 送 “ 近 端 ” 、 “ 远 端 ” 及 环 周 底 部 切 除 面 未 查 见 癌 。 肠 壁 一 站 （ 1 0 个 ） 、 中 间 组 （ 8 个 ） 淋 巴 结 未 查 见 癌 。 ， 免 疫 组 化 染 色 示 ： E R C C 1 弥 漫 （ + ） 、 T S 少 部 分 弱 （ + ） 、 S Y N （ - ） 、 C G A （ - ） 。 术 后 查 无 化 疗 禁 忌 后 给 予 3 周 期 化 疗 ， ， 方 案 为 ： 奥 沙 利 铂 1 5 0 M G O D 1 ， 亚 叶 酸 钙 0 . 3 G + 替 加 氟',
 [101,
  8024,
  2642,
  5442,
  124,
  3299,
  1184,
  1728,
  100,
  4684,
  5499,
  4617,
  100,
  754,
  1762,
  2769,
  7368,
  754,
  1059,
  7937,
  677,
  6121,
  4684,
  5499,
  4617,
  3418,
  3780,
  3318,
  8020,
  146,
  151,
  166,
  157,
  156,
  3318,
  8021,
  8024,
  2797,
  3318,
  6814,
  4923,
  7556,
  1164,
  8024,
  3318,
  1400,
  5314,
  750,
  2834,
  2697,
  3381,
  1350,
  5852,
  1075,
  3118,
  2898,
  3780,
  4545,
  8024,
  2642,
  5442,
  261

In [9]:
train_dataset[1]

('[CLS] ， 患 者 因 罹 患 “ 胃 癌 ” 于 2 0 1 3 - 1 0 - 2 9 在 我 院 予 行 全 麻 上 胃 癌 根 治 术 ， ， 术 中 见 ： 腹 腔 内 腹 水 ， 腹 膜 无 转 移 ， 肝 脏 未 触 及 明 显 转 移 性 灶 ， 肿 瘤 位 于 胃 体 、 胃 底 部 ， 小 弯 侧 偏 后 壁 ， 约 5 * 4 * 2 C M 大 小 ， 肿 瘤 已 侵 达 浆 膜 外 ， 第 1 、 3 组 淋 巴 结 肿 大 ， 肿 瘤 尚 能 活 动 ， 经 探 查 决 定 行 全 胃 切 除 ， 空 肠 J 字 代 胃 术 。 手 术 顺 利 ， 术 后 积 极 予 相 关 对 症 支 持 治 疗 ； ， 后 病 理 示 ： 胃 底 、 体 小 弯 侧 低 分 化 腺 癌 ， 部 分 为 印 戒 细 胞 癌 图 像 ， 蕈 伞 型 ， 面 积 5 . 2 * 3 . 5 C M ， 局 部 侵 达 粘 膜 上 层 ， 并 于 少 数 腺 管 内 查 见 癌 栓 。 双 端 切 线 及 另 送 “ 近 端 切 线 ” 未 查 见 癌 。 呈 三 组 （ 5 / 1 3 个 ） 淋 巴',
 [101,
  8024,
  2642,
  5442,
  1728,
  5395,
  2642,
  100,
  5517,
  4617,
  100,
  754,
  123,
  121,
  122,
  124,
  118,
  122,
  121,
  118,
  123,
  130,
  1762,
  2769,
  7368,
  750,
  6121,
  1059,
  7937,
  677,
  5517,
  4617,
  3418,
  3780,
  3318,
  8024,
  8024,
  3318,
  704,
  6224,
  8038,
  5592,
  5579,
  1079,
  5592,
  3717,
  8024,
  5592,
  5606,
  3187,
  6760,
  4919,
  8024,
  5498,
  5552,
  3313,
  6239,
  1350,
  3209,
  3227,
  6760,
  4919,
 

### 分段处理，使长度低于256

In [37]:
MAX_LEN = 256 - 2
sents, tags_li = [], [] # list of lists
for entry in entries:
    words = [line.split()[0] for line in entry.splitlines()]
    tags = ([line.split()[-1] for line in entry.splitlines()])
#     sents.append(["[CLS]"] + words + ["[SEP]"])  # 每个句子前后加['CLS']和['SEP']
#     tags_li.append(["<PAD>"] + tags + ["<PAD>"])
    if len(words) > MAX_LEN:
        # 先对句号分段
        word, tag = [], []
        for char, t in zip(words, tags):
            
            if char != '。':
                word.append(char)
                tag.append(t)
            else:
#                 if len(word) > MAX_LEN:
#                     word = word[:MAX_LEN]
#                     tag = tag[:MAX_LEN]
                sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
                tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
                word, tag = [], []            
        # 最后的
        if len(word):
            sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
            tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
            word, tag = [], []
    else:
        sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
        tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])

In [38]:
len(sents)

6306

In [39]:
len(tags_li)

6306

In [36]:
sents[0], tags_li[0]

(['，',
  '患',
  '者',
  '3',
  '月',
  '前',
  '因',
  '“',
  '直',
  '肠',
  '癌',
  '”',
  '于',
  '在',
  '我',
  '院',
  '于',
  '全',
  '麻',
  '上',
  '行',
  '直',
  '肠',
  '癌',
  '根',
  '治',
  '术',
  '（',
  'D',
  'I',
  'X',
  'O',
  'N',
  '术',
  '）',
  '，',
  '手',
  '术',
  '过',
  '程',
  '顺',
  '利',
  '，',
  '术',
  '后',
  '给',
  '予',
  '抗',
  '感',
  '染',
  '及',
  '营',
  '养',
  '支',
  '持',
  '治',
  '疗',
  '，',
  '患',
  '者',
  '恢',
  '复',
  '好',
  '，',
  '切',
  '口',
  '愈',
  '合',
  '良',
  '好',
  '。',
  '，',
  '术',
  '后',
  '病',
  '理',
  '示',
  '：',
  '直',
  '肠',
  '腺',
  '癌',
  '（',
  '中',
  '低',
  '度',
  '分',
  '化',
  '）',
  '，',
  '浸',
  '润',
  '溃',
  '疡',
  '型',
  '，',
  '面',
  '积',
  '3',
  '.',
  '5',
  '*',
  '2',
  'C',
  'M',
  '，',
  '侵',
  '达',
  '外',
  '膜',
  '。',
  '双',
  '端',
  '切',
  '线',
  '另',
  '送',
  '“',
  '近',
  '端',
  '”',
  '、',
  '“',
  '远',
  '端',
  '”',
  '及',
  '环',
  '周',
  '底',
  '部',
  '切',
  '除',
  '面',
  '未',
  '查',
  '见',
  '癌',
  '。',
  '肠',
  '壁',
  '一',
  '站',

In [2]:
from pytorch_pretrained_bert import BertModel
bert = BertModel.from_pretrained('bert-base-chinese')

In [4]:
x = dev_dataset[1]

In [9]:
import torch
encode_layers, _ = bert(torch.tensor(x[1]).unsqueeze(0))
enc = encode_layers[-1]
enc

tensor([[[-0.4081,  0.6838, -0.1342,  ..., -0.6718, -0.5864,  0.2491],
         [-0.1821,  0.4146,  0.9377,  ..., -1.1175, -0.4919,  0.0507],
         [-0.2332,  0.3116,  0.1658,  ...,  0.4438,  0.3029, -0.0686],
         ...,
         [-0.0225, -0.1823, -0.4035,  ...,  0.8034, -0.0065, -0.0556],
         [-0.5497,  0.5112,  1.1299,  ...,  0.3418, -0.0035, -0.6358],
         [-0.2669, -0.1594, -0.0355,  ...,  0.3744, -0.3602, -0.0103]]],
       grad_fn=<AddBackward0>)

In [10]:
enc.size()

torch.Size([1, 59, 768])

In [11]:
len(x[1])

59

In [14]:
enc = enc.view(len(x[1]), 1, -1)
enc.size()

torch.Size([59, 1, 768])